In [3]:
from datasets import load_dataset
import pandas as pd

# First time run this code, and it will download a json file 
dataset = load_dataset("taddeusb90/finbro-v0.1.0", cache_dir="../data")
dataset = pd.DataFrame(dataset)
dataset = pd.json_normalize(dataset['train'])

Generating train split: 100%|██████████| 384058/384058 [00:01<00:00, 309274.34 examples/s]


In [4]:
print(dataset)

                                                    input  \
0       Residual Standard Deviation: Definition, Formu...   
1             How to Use a Home Equity Loan for a Remodel   
2                Microsoft (MSFT) Unveils Space Ambitions   
3                                Crime of 1873 Definition   
4       What Is a Capital Asset? How It Works, With Ex...   
...                                                   ...   
384053  You are a financial analyst categorizing tweet...   
384054  You are a financial analyst categorizing tweet...   
384055  You are a financial analyst categorizing tweet...   
384056  You are a financial analyst categorizing tweet...   
384057  You are a financial analyst categorizing tweet...   

                                              instruction  \
0        In the given passage, how are residual values...   
1        What are some advantages of using a home equi...   
2        Which companies has AWS partnered with to bro...   
3        When was the a